# Linear regression

We estimate simple linear regression model with a half-T prior.
First, we load the packages we use.

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

Import the dataset.

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults and standardize

In [3]:
df2 = filter(row -> row[:age] >= 18, df)

,height,weight,age,male
,Float64⍰,Float64⍰,Float64⍰,Int64⍰
1,151.765,47.8256,63.0,1
2,139.7,36.4858,63.0,0
3,136.525,31.8648,65.0,0
4,156.845,53.0419,41.0,1
5,145.415,41.2769,51.0,0
6,163.83,62.9926,35.0,1
7,149.225,38.2435,32.0,0
8,168.91,55.48,27.0,1
9,147.955,34.8699,19.0,0


Show the first six rows of the dataset.

In [4]:
first(df2, 6)

,height,weight,age,male
,Float64⍰,Float64⍰,Float64⍰,Int64⍰
1,151.765,47.8256,63.0,1
2,139.7,36.4858,63.0,0
3,136.525,31.8648,65.0,0
4,156.845,53.0419,41.0,1
5,145.415,41.2769,51.0,0
6,163.83,62.9926,35.0,1


Then define a structure to hold the data: observables and the degrees of freedom for the prior.

In [5]:
"""
Half-T for `σ`.
"""
struct HeightsProblem{TY <: AbstractVector, Tν <: Real}
    "Observations."
    y::TY
    "Degrees of freedom for prior on sigma."
    ν::Tν
end

Main.##402.HeightsProblem

Then make the type callable with the parameters *as a single argument*.

In [6]:
function (problem::HeightsProblem)(θ)
    @unpack y, ν = problem   # extract the data
    @unpack μ, σ = θ
    loglikelihood(Normal(μ, σ), y) + logpdf(TDist(ν), σ)
end

We should test this, also, this would be a good place to benchmark and
optimize more complicated problems.

In [7]:
obs = convert(Vector{Float64}, df2[:height])
p = HeightsProblem(obs, 1.0);
p((μ = 178, σ = 5.0,))

-5170.976519811121

Wrap the problem with a transformation, then use Flux for the gradient.

In [8]:
P = TransformedLogDensity(as((σ = as(Real, 0, 10), μ  = as(Real, 0, 200))), p)
∇P = ADgradient(:ForwardDiff, P);

Finally, we sample from the posterior. `chain` holds the chain (positions and
diagnostic information), while the second returned value is the tuned sampler
which would allow continuation of sampling.

In [9]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0015 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00021 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0022 s/step ...done
MCMC, adapting ϵ (100 steps)
8.0e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
5.7e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
4.1e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
7.4e-5 s/step ...done
MCMC (1000 steps)
5.7e-5 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [10]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));

Extract the parameter posterior means: `β`,

In [11]:
posterior_β = mean(first, posterior)

7.731627250863755

then `σ`:

In [12]:
posterior_σ = mean(last, posterior)

154.59421123583394

Effective sample sizes (of untransformed draws)

In [13]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)
# NUTS-specific statistics
NUTS_statistics(chain)

cmdstan_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD       Naive SE      MCSE      ESS
sigma   7.7641872 0.29928194 0.004732063 0.0055677898 1000
   mu 154.6055177 0.41989355 0.006639100 0.0085038356 1000

Quantiles:
         2.5%      25.0%       50.0%      75.0%       97.5%
sigma   7.21853   7.5560625   7.751355   7.9566775   8.410391
   mu 153.77992 154.3157500 154.602000 154.8820000 155.431000
";

Extract the parameter posterior means: `β`,

In [14]:
[posterior_β, posterior_σ]

2-element Array{Float64,1}:
   7.731627250863755
 154.59421123583394 

end of m4.5d.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*